In [1]:
import common
import importlib
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import socket

# Show all columns and rows in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# graphalytics inserts
pers_experiments = "num_threads_read == 0 and (hostname == 'scyper15' or hostname == 'scyper22' or hostname == 'scyper21')"

data_ins = common.import_gfe("view_graphalytics_inserts").query(pers_experiments).copy() # data from the experiments
data_ins["build_frequency"].fillna(pd.Timedelta(0), inplace=True) # replace NaT with 0, otherwise the records are ignored in the group by

data_ins = data_ins.query("((block_size == 512) and library == 'sortledton.1') or library != 'sortledton.1'")

# graphalytics load
data_load = common.import_gfe("view_graphalytics_load").query(pers_experiments).copy()
# shape the dataframe as data_ins
data_load["build_frequency"] = pd.Timedelta(0)
data_load["build_frequency_secs"] = np.NaN
data_load["num_snapshots_created"] = 0
data_load["num_threads_write"] = 0
data_load["block_size"] = -1

data = pd.concat([data_ins, data_load])
data["validate_inserts"].fillna("0", inplace=True)
data["validate_output"].fillna("0", inplace=True)


In [3]:
#data = data.query("validate_inserts == '1' and validate_output == '1' and library == 'sortledton.1'")

data = data.groupby(["library", "graph"]) \
    .agg(validate_inserts=("validate_inserts", "max"), validate_outputs=("validate_output", "max"), validation_errors=("num_validation_errors", "max"))
data

validate_inserts  \
library                             graph                             
baseline_v3                         dota-league                   0   
                                    graph500-24                   0   
                                    uniform-24                    0   
csr3                                com-friendster                0   
                                    dota-league                   0   
                                    graph500-22                   0   
                                    graph500-24                   0   
                                    graph500-26                   0   
                                    uniform-24                    0   
                                    uniform-26                    0   
csr3-lcc                            com-friendster                0   
                                    dota-league                   0   
                                    graph500-22                   0   
                                    graph500-24                   1   
                                    graph500-26                   0   
                                    uniform-24                    1   
                                    uniform-26                    0   
csr3-lcc-numa                       com-friendster                0   
                                    dota-league                   1   
                                    graph500-22                   1   
                                    graph500-24                   1   
                                    graph500-26                   1   
                                    uniform-24                    1   
                                    uniform-26                    1   
csr4-lcc                            graph500-24                   0   
edgeiter_sorted_vector_al           dota-league                   1   
edgeiter_sorted_vector_al.1         com-friendster                0   
                                    dota-league                   0   
                                    graph500-24                   0   
edgeiter_sorted_vector_al.2         dota-league                   0   
                                    graph500-24                   0   
                                    uniform-24                    0   
edgeiter_sorted_vector_al.3         com-friendster                0   
                                    dota-league                   0   
                                    graph500-24                   0   
                                    uniform-24                    0   
g1_v6-ref-ignore-build              dota-league                   1   
                                    graph500-22                   1   
                                    graph500-24                   1   
                                    graph500-26                   1   
                                    uniform-24                    1   
                                    uniform-26                    1   
livegraph3_ro                       com-friendster                0   
                                    dota-league                   1   
                                    graph500-22                   1   
                                    graph500-24                   1   
                                    graph500-26                   1   
                                    uniform-24                    1   
                                    uniform-26                    1   
llama8-ref                          dota-league                   1   
                                    graph500-22                   1   
                                    graph500-24                   1   
                                    uniform-24                    1   
mb-csr.3                            dota-league                   1   
                                    graph500-24                   0   
    

All data

In [4]:
data_missing = data.query("validate_inserts == '0' or validate_outputs == '0'")
old_libraries = ["baseline_v3", "sorted_vector_al.1", "sorted_vector_al", "vector_al", "vector_al.1", "vector_al.2", "sortledton", "tree_sorted_vector_al", "tree_sorted_vector_al.1", "tree_sorted_vector_al.2", "robin_hood_al.2", "edgeiter_sorted_vector_al.2", "edgeiter_sorted_vector_al.1", "robin_hood_al.1", "robin_hood_sorted_vector_al.2", "robin_hood_sorted_vector_al.1", "robin_hood_sorted_vector_al"]
libraries_query = " and ".join(list(map(lambda l: "library != '%s'" % l, old_libraries)))
data_missing = data_missing.query(libraries_query)
data_missing

validate_inserts  \
library                             graph                             
csr3                                com-friendster                0   
                                    dota-league                   0   
                                    graph500-22                   0   
                                    graph500-24                   0   
                                    graph500-26                   0   
                                    uniform-24                    0   
                                    uniform-26                    0   
csr3-lcc                            com-friendster                0   
                                    dota-league                   0   
                                    graph500-22                   0   
                                    graph500-26                   0   
                                    uniform-26                    0   
csr3-lcc-numa                       com-friendster                0   
csr4-lcc                            graph500-24                   0   
edgeiter_sorted_vector_al.3         com-friendster                0   
                                    dota-league                   0   
                                    graph500-24                   0   
                                    uniform-24                    0   
livegraph3_ro                       com-friendster                0   
mb-csr.3                            graph500-24                   0   
                                    uniform-24                    0   
mb-csr.5                            dota-league                   0   
                                    graph500-24                   0   
                                    uniform-24                    0   
mb-csr.6                            graph500-22                   0   
                                    graph500-26                   0   
                                    uniform-26                    0   
mb-csr.7                            graph500-24                   0   
robin_hood_al.3                     com-friendster                0   
                                    dota-league                   0   
                                    graph500-24                   0   
                                    uniform-24                    0   
robin_hood_sorted_vector_al.3       com-friendster                0   
                                    dota-league                   0   
                                    graph500-24                   0   
                                    uniform-24                    0   
single-numa-node-csr3-lcc           graph500-24                   0   
                                    uniform-24                    0   
single-numa-node-sorted-vector_al.2 graph500-24                   0   
                                    uniform-24                    0   
sorted_vector_al.2                  graph500-24                   0   
                                    uniform-24                    0   
sorted_vector_al.3                  com-friendster                0   
                                    graph500-24                   0   
                                    uniform-24                    0   
sorted_vector_al.5                  graph500-24                   0   
                                    uniform-24                    0   
teseo-lcc.12                        com-friendster                0   
tree_sorted_vector_al.3             com-friendster                0   
                                    dota-league                   0   
                                    graph500-24                   0   
                                    uniform-24                    0   
vector_al.3                         com-friendster                0   
                                    graph500-24                   0   
                                    uniform-24                    0   

   

In [5]:
data_error = data.query("validation_errors != 0 and validation_errors != -1 and validation_errors == validation_errors")
data_error

,,validate_inserts,validate_outputs,validation_errors
library,graph,,,
